# **Regression**

## Objectives

*   Fit and evaluate a regression model to predict Wing Span for Single piston Engined airplanes.


## Inputs

* outputs/datasets/collection/airplane_performance_study.csv
* Instructions on which variables to use for data cleaning and feature engineering. They are found in their respective notebooks.

## Outputs

* feature_importance (3 files) 
* Mean Squared Error
* Predicted vs Actual Plot
* Residuals Distribution Plot
* Residuals vs Fitted Plot
* Train set
* Test set
* Modeling pipeline

---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* Access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Data

In order to not compare apples with oranges the code below filters out all Multi Engined, jet and propjet leaving only the subset of **single Piston Engines**. It drops these columns as well as the performance modification (TP mods) column and the Hmax_(One) and ROC_(One)-columns since they only pertain to Multi Engined airplane.

In [ ]:
import numpy as np
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/airplane_performance_study.csv")
      .query("Engine_Type == 0 and Multi_Engine == 0 and TP_mods == 0")  # subset of airplanes with single Piston Enginees without any Turbo Prop performance modification TP_mods dmultiple Engines
      .drop(labels=['Model', 'Company', 'THR', 'SHP', 'Engine_Type', 'Multi_Engine', 'TP_mods', 'Hmax_(One)', 'ROC_(One)'], axis=1)
      )

print(df.shape)
df.head(10)

---

## Define features and target

Define features and target

X are the features for the model (all columns in df except for target Wing_Span.
y is the target variable which is the Wing_Span column in the df.

In [5]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Define features and target
X = df.drop('Wing_Span', axis=1)  # Features (where I drop the target variable)
y = df['Wing_Span']                # Target variable

## Split Train Test Set

80% of df is split into the train set and 20% is left for the test set.
The random_state=0 parameter ensures reproducibility of the split.

We see that we have a suffiecent numper of data points to work with.

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  # 80% to train and 20% to test.

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",  X_test.shape, y_test.shape)

---

# MP Pipeline: Regressor

## Create Pipeline

'scaler' applies standard scaling to the (numerical) features, ensuring that they have a mean = 0 and a standard deviation = 1.
'regressor' initializes Random Forest regression model for predicting target variable (Wing_Span).

In [7]:
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),                     # Scale numerical features
    ('regressor', RandomForestRegressor(random_state=0))  # Regression model using Random Forrest
])

## Fit the model

build the training model using the training dataset.

StandardScaler is applied to scale the features in X_train.

RandomForestRegressor is trained on the scaled features (from previous step) and corresponding target values.

In [ ]:
pipeline.fit(X_train, y_train)

## Make predictions

Scaling test features using StandardScaler fitted during training.

Using the trained RandomForestRegressor to predict on scaled test data.

Output: The predicted values for target (Wing_Span) stored in the predictions variable.

In [9]:
predictions = pipeline.predict(X_test)

---

## Evaluate Model Test Set

### Mean Squared Error Calculation

The mean_squared_error computes the mean squared error (MSE) between the actual values (y_test) and the predicted values (predictions). The MSE provids a quantitative measure of the model's performance on the test dataset.

 MSE is ok but not great, typically off by almost 3 ft (square root on MSE)!

In [ ]:
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

Save above output

In [11]:
# Save MSE to a text file
with open('outputs/ml_pipeline/predict_analysis/mse.txt', 'w') as f:
    f.write(f'Mean Squared Error: {mse}\n')

## Print feature importances (for RandomForestRegressor)

feature_importance_df is created that in descending order (by importance) list an "importance score"-for each feature providing insights into which features are most influential in predicting the target variable (Wing_Span).It makes much sense that the weights are on top since the Wing Span is related to the Wing Area (not in the data set) and that Wing Area is directly related to lift that in turn is directly related to the weights (since the lift need to offset the weight in order for the airplane to fly)

In [ ]:
importances = pipeline.named_steps['regressor'].feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
print(feature_importance_df.sort_values(by='Importance', ascending=False))

Save above output

In [ ]:
import joblib

# Save the feature importances as a .pkl file
joblib.dump(importances, 'outputs/ml_pipeline/predict_analysis/feature_importances.pkl')

# Save the feature names as a .pkl file
joblib.dump(feature_names.tolist(), 'outputs/ml_pipeline/predict_analysis/feature_names.pkl')

# Save the feature importance DataFrame as a .pkl file
joblib.dump(feature_importance_df, 'outputs/ml_pipeline/predict_analysis/feature_importance_df.pkl')


---

## Plot Regression

### Predicted vs actual values
The points in the scatter plot represent the predicted Wing_Span values (on the y-axis) against the actual Wing_Span values (on the x-axis) for the test set.

In [ ]:
print(y_test.shape)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
plt.scatter(y_test, predictions, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # Diagonal line
plt.title('Predicted vs Actual Wing Span')
plt.xlabel('Actual Wing Span')
plt.ylabel('Predicted Wing Span')
plt.xlim([y_test.min(), y_test.max()])
plt.ylim([y_test.min(), y_test.max()])
plt.grid()
plt.savefig('outputs/ml_pipeline/predict_analysis/predicted_vs_actual.png')  # Save the plot as a PNG file
plt.show()

### Residual plot

Calculating Residuals: The residuals are calculated by subtracting the predicted values (predictions) from the actual values (y_test). Residuals indicate the difference between what the model predicted and what the actual values were.

This histogram helps visualize the distribution of the residuals, which can indicate how well the model is performing. Ideally, residuals should be normally distributed around zero if the model is a good fit.

The Residuals Distribution shows that the distribution of errors (if negelecting the outlier) are such that all errors are all well within +- 5 meter. (and that the model rather tend overestimate than underestimate the wingspan).

In [ ]:
residuals = y_test - predictions
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True)
plt.title('Residuals Distribution')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.grid()
plt.savefig('outputs/ml_pipeline/predict_analysis/residuals_distribution.png')  # Save the plot as a PNG file
plt.show()

### Residuals vs Fitted plot

Each point represents a fitted value (predicted wingspan) from the train set plotted against its corresponding residual.

Ideally, the residuals should be randomly scattered around the horizontal line at zero without any discernible pattern however our pattern appear to be "high" which is consistent with our previous plots.

If we the "High pattern" is real it could indicate: it may indicate issues such as non-linearity, omitted variables, or model miss-specification.

We also notice an outlier at the bottom of the graf that we could hunt down and remove since it potentially could skew the data.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(predictions, residuals, alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Fitted')
plt.xlabel('Fitted Values (Predicted Wing Span)')
plt.ylabel('Residuals')
plt.grid()
plt.savefig('outputs/ml_pipeline/predict_analysis/residuals_vs_fitted.png')  # Save the plot
plt.show()


---

## A second Evaluation of model based on observation in plots

All previous plots have appearntly given a slight overestimation of the Wingspan. This is confirmed by a 
Mean Error (ME) of approximately 5%. The Relative Error of 0.18% (based on a Wingspan range for the y_test of 29.08 feet) shows that this is error is not significant.

In [ ]:
mean_error = np.mean(predictions - y_test)
print("Mean Error (ME):", mean_error)

range_value = y_test.max() - y_test.min()
print(f"Range of y_test: {range_value:.2f} feet")

# Calculate the relative error
relative_error = mean_error / range_value
print(f"Relative Error: {relative_error:.4f} ({relative_error * 100:.2f}%)")

# Save the results to a text file
output_path = 'outputs/ml_pipeline/predict_analysis/error_analysis.txt'
with open(output_path, 'w') as f:
    f.write(f'Mean Error (ME): {mean_error:.4f} feet\n')
    f.write(f'Range of y_test: {range_value:.2f} feet\n')
    f.write(f'Relative Error: {relative_error:.4f} ({relative_error * 100:.2f}%)\n')

---

# Push files to the repo

The following plots has already been save upstream right after they where created:
* feature_importance (3 files) 
* Mean Squared Error
* Predicted vs Actual Plot
* Residuals Distribution Plot
* Residuals vs Fitted Plot

Below we will generate the following files:
* Train set
* Test set
* Modeling pipeline

In [ ]:
joblib.dump(pipeline, 'outputs/ml_pipeline/predict_analysis/wingspan_predictor_model.pkl')


In [ ]:
import joblib
import os

file_path = f'outputs/ml_pipeline/predict_analysis'

try:
  os.makedirs(name=file_path)
except Exception as e:
  print(e)

## Train Set: features and target

In [ ]:
X_train.head()

In [22]:
X_train.head().to_csv('outputs/ml_pipeline/predict_analysis/X_train_head.csv', index=False)

In [ ]:
y_train

In [24]:
y_train.to_csv('outputs/ml_pipeline/predict_analysis/y_train.csv', index=False)

## Test Set: features and target

In [ ]:
X_test.head()

In [26]:
X_test.head().to_csv('outputs/ml_pipeline/predict_analysis/X_test_head.csv', index=False)

In [ ]:
y_test

In [28]:
y_test.to_csv('outputs/ml_pipeline/predict_analysis/y_test.csv', index=False)

---